#Projet Python M2

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
import pickle


In [2]:
try:
  df_merged = pd.read_csv("../data/merged_69.csv", sep=";") # chemin spécifier
  #print(df_merged.columns)
except FileNotFoundError:
  print("Fichier non trouvé")

C:\Users\lcisse\AppData\Local\Temp\ipykernel_51812\4243165685.py:2: DtypeWarning: Columns (33,35,38,51,57,69,72,99,115) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merged = pd.read_csv("../data/merged_69.csv", sep=";") # chemin spécifier


In [3]:
# MLFlow ?
df_merged.head()

,Ubat_W/m²_K,Complément_d'adresse_bâtiment,Méthode_application_DPE,N°_département_(BAN),Qualité_isolation_menuiseries,Nombre_appartement,Emission_GES_5_usages_énergie_n°2,Coordonnée_cartographique_X_(BAN),Date_fin_validité_DPE,Protection_solaire_exterieure_(0/1),...,Modèle_DPE,Coût_auxiliaires,Coût_refroidissement_dépensier,Nom__rue_(BAN),Nom__commune_(BAN),Emission_GES_ECS_dépensier,Conso_5_usages_par_m²_é_primaire,Besoin_chauffage,Appartement_non_visité_(0/1),Complément_d'adresse_logement
0,1.42,NaN,dpe appartement individuel,69.0,moyenne,1.0,NaN,842365.11,2034-08-28,False,...,DPE 3CL 2021 méthode logement,0.0,0.0,Rue Imbert Colomès,Lyon,117.1,535.0,5261.0,False,1-appartement-1-LogZone
1,2.36,653199,dpe appartement individuel,69.0,moyenne,1.0,NaN,842432.78,2034-07-23,False,...,DPE 3CL 2021 méthode logement,68.6,0.0,Rue Longue,Lyon,107.4,583.0,9089.0,NaN,2ième Etage Porte Gauche
2,0.59,2024-08-15639,dpe appartement individuel,69.0,moyenne,1.0,16.7,842540.42,2034-08-11,True,...,DPE 3CL 2021 méthode logement,62.5,0.0,Rue Imbert Colomès,Lyon,387.9,386.0,8784.5,NaN,Etage 6
3,0.42,23/DFA/0199,dpe appartement individuel,69.0,moyenne,1.0,34.5,842600.66,2034-07-28,True,...,DPE 3CL 2021 méthode logement,135.1,0.0,Rue Romarin,Lyon,465.0,156.0,5039.0,NaN,NaN
4,1.29,240911838,dpe appartement individuel,69.0,bonne,1.0,NaN,842544.94,2034-09-09,False,...,DPE 3CL 2021 méthode logement,19.6,0.0,Rue Saint Polycarpe,Lyon,75.8,152.0,779.6,NaN,Etage 6


In [4]:
df_merged.shape

(287165, 131)

In [5]:
# Afficher la liste complète des noms de colonnes pour identifier les cibles "étiquettes DPE" et "consommation énergétique"
columns = df_merged.columns.tolist()
columns

['Ubat_W/m²_K',
 "Complément_d'adresse_bâtiment",
 'Méthode_application_DPE',
 'N°_département_(BAN)',
 'Qualité_isolation_menuiseries',
 'Nombre_appartement',
 'Emission_GES_5_usages_énergie_n°2',
 'Coordonnée_cartographique_X_(BAN)',
 'Date_fin_validité_DPE',
 'Protection_solaire_exterieure_(0/1)',
 'Coût_total_5_usages',
 'Conso_ECS_dépensier_é_primaire',
 'Catégorie_ENR',
 'Présence_brasseur_air_(0/1)',
 'Date_établissement_DPE',
 'Score_BAN',
 'Type_énergie_n°2',
 'Conso_auxiliaires_é_primaire',
 'N°_étage_appartement',
 'N°_région_(BAN)',
 'Deperditions_planchers_hauts',
 'Adresse_brute',
 'Emission_GES_refroidissement_dépensier',
 'Version_DPE',
 'Deperditions_planchers_bas',
 'Surface_tertiaire_immeuble',
 'Coût_chauffage',
 '_i',
 'Qualité_isolation_plancher_bas',
 'Déperditions_renouvellement_air',
 'Surface_habitable_logement',
 'Type_énergie_n°1',
 'Conso_chauffage_é_finale',
 'Besoin_refroidissement',
 'Emission_GES_éclairage',
 'Code_INSEE_(BAN)',
 'Conso_refroidissement_

In [6]:
df1 = df_merged

In [7]:
df1.shape

(287165, 131)

In [8]:
pd.set_option('display.max_rows', None)

In [9]:
#Fonction pour identifier le % de valeurs manquantes dans chaque colonne
#Renvoie seulement les colonnes contenant des valeurs manquantes + les tri
def percent_missing(df):
  percent_nan = 100 * df.isnull().sum() / len(df)
  percent_nan = percent_nan[percent_nan > 0].sort_values()

  return percent_nan

percent_nan = percent_missing(df1)
percent_nan

Ubat_W/m²_K                                        0.000348
Conso_auxiliaires_é_finale                         0.000348
Conso_chauffage_dépensier_é_finale                 0.002089
Conso_chauffage_é_finale                           0.002089
Emission_GES_chauffage                             0.002089
Conso_chauffage_dépensier_é_primaire               0.002089
Emission_GES_chauffage_dépensier                   0.002089
Conso_chauffage_é_primaire                         0.002089
Coût_éclairage                                     0.002438
Coût_refroidissement                               0.002786
Emission_GES_auxiliaires                           0.002786
Conso_5_usages_é_primaire                          0.002786
Emission_GES_5_usages_par_m²                       0.002786
Conso_5_usages_é_finale                            0.002786
Emission_GES_5_usages_énergie_n°1                  0.002786
Conso_auxiliaires_é_primaire                       0.002786
Conso_5_usages/m²_é_finale              

In [10]:
#suppression des colonnes qui ont + de > 30% valeurs manquantes
df1 = df1.drop(percent_nan[percent_nan > 30].index, axis=1)

In [11]:
percent_nan = percent_missing(df1)
percent_nan

Ubat_W/m²_K                              0.000348
Conso_auxiliaires_é_finale               0.000348
Conso_chauffage_dépensier_é_finale       0.002089
Conso_chauffage_é_finale                 0.002089
Conso_chauffage_dépensier_é_primaire     0.002089
Emission_GES_chauffage_dépensier         0.002089
Emission_GES_chauffage                   0.002089
Conso_chauffage_é_primaire               0.002089
Coût_éclairage                           0.002438
Coût_refroidissement                     0.002786
Conso_5_usages_é_finale                  0.002786
Conso_5_usages_é_primaire                0.002786
Emission_GES_5_usages_énergie_n°1        0.002786
Emission_GES_5_usages_par_m²             0.002786
Conso_auxiliaires_é_primaire             0.002786
Emission_GES_auxiliaires                 0.002786
Coût_chauffage_énergie_n°1               0.002786
Conso_5_usages_é_finale_énergie_n°1      0.002786
Coût_auxiliaires                         0.002786
Conso_5_usages_par_m²_é_primaire         0.002786


In [12]:
#supprimer les lignes, dans les colonnes qui ont moins de 9% de valeurs manquantes
df1 = df1.dropna(subset=percent_nan[percent_nan < 9].index, axis=0)

In [13]:
percent_nan = percent_missing(df1)
percent_nan

Deperditions_planchers_bas          0.842587
Deperditions_planchers_hauts        0.842587
Déperditions_ponts_thermiques       0.842587
Déperditions_renouvellement_air     0.842587
Deperditions_enveloppe              0.842587
Déperditions_murs                   0.842587
Deperditions_baies_vitrées          0.842587
Déperditions_portes                 0.842587
Besoin_refroidissement              0.862077
Besoin_ECS                          0.862077
Besoin_chauffage                    0.862077
Classe_inertie_bâtiment             0.863890
N°_voie_(BAN)                       5.581290
N°_étage_appartement                8.741332
Type_installation_chauffage        11.077369
Type_installation_ECS_(général)    11.077369
Nombre_appartement                 14.385170
Complément_d'adresse_logement      22.561755
dtype: float64

In [14]:
'''#Imputation des Données Manquantes sur variable quantitatives
import numpy as np
from sklearn.impute import KNNImputer
# Sélectionner uniquement les colonnes quantitatives (numériques)
quant_cols = df1.select_dtypes(include=[np.number]).columns
# Afficher les colonnes quantitatives
print("Colonnes quantitatives :", quant_cols)
# Initialiser le KNNImputer
imputer = KNNImputer(n_neighbors=3)
# Appliquer l'imputation sur les colonnes quantitatives
cols_with_missing = [col for col in quant_cols if df1[col].isnull().any()]
df1[cols_with_missing] = imputer.fit_transform(df1[cols_with_missing])
# Vérification des données manquantes
#print(df1[quant_cols].isnull().sum())
'''

'#Imputation des Données Manquantes sur variable quantitatives\nimport numpy as np\nfrom sklearn.impute import KNNImputer\n# Sélectionner uniquement les colonnes quantitatives (numériques)\nquant_cols = df1.select_dtypes(include=[np.number]).columns\n# Afficher les colonnes quantitatives\nprint("Colonnes quantitatives :", quant_cols)\n# Initialiser le KNNImputer\nimputer = KNNImputer(n_neighbors=3)\n# Appliquer l\'imputation sur les colonnes quantitatives\ncols_with_missing = [col for col in quant_cols if df1[col].isnull().any()]\ndf1[cols_with_missing] = imputer.fit_transform(df1[cols_with_missing])\n# Vérification des données manquantes\n#print(df1[quant_cols].isnull().sum())\n'

In [15]:
#Imputation des Données Manquantes sur variable qualitative

# Sélectionner toutes les colonnes non numériques (qualitatives)
categorical_cols = df1.select_dtypes(exclude=[np.number]).columns

# Appliquer l'imputation par la valeur la plus fréquente (mode) pour chaque colonne catégorielle
for col in categorical_cols:
    df1[col] = df1[col].fillna(df1[col].mode()[0])

# Vérification des données manquantes
#df1[categorical_cols].isnull().sum()

In [16]:
col = df1.columns.tolist()
col
# Sélectionner toutes les colonnes sauf celles contenant "Coût" ou "Conso" ou "GES" qui sont liées a la variable target pour éviter Data Leakage
les_variables_explicatives = [col for col in df1.columns if not ("Coût" in col or "Conso" in col or "GES" in col or "coût" in col)]
#les_variables_explicatives

In [17]:
# Filtrer les colonnes quantitatives parmi les variables explicatives
quantitative_variables = [col for col in les_variables_explicatives if np.issubdtype(df1[col].dtype, np.number)]
# Afficher les variables quantitatives sélectionnées
#print("Variables quantitatives parmi les variables explicatives :", quantitative_variables)
quantitative_variables

['Ubat_W/m²_K',
 'N°_département_(BAN)',
 'Nombre_appartement',
 'Coordonnée_cartographique_X_(BAN)',
 'Score_BAN',
 'N°_étage_appartement',
 'N°_région_(BAN)',
 'Deperditions_planchers_hauts',
 'Version_DPE',
 'Deperditions_planchers_bas',
 '_i',
 'Déperditions_renouvellement_air',
 'Surface_habitable_logement',
 '_score',
 'Nombre_niveau_logement',
 'Déperditions_ponts_thermiques',
 'Coordonnée_cartographique_Y_(BAN)',
 'Deperditions_enveloppe',
 'Hauteur_sous-plafond',
 'Déperditions_portes',
 'Code_postal_(BAN)',
 '_rand',
 'Déperditions_murs',
 'Deperditions_baies_vitrées',
 'Code_postal_(brut)',
 'Besoin_chauffage']

In [18]:
# Supprimer les colonnes constantes (qui ne donneront donc aucune corrélation)
quantitative_variables = [col for col in quantitative_variables if df1[col].nunique() > 1]
# Calculer la corrélation entre chaque variable quantitative et 'Conso_5_usages_é_finale'
correlations = df1[quantitative_variables].corrwith(df1['Conso_5_usages_é_finale'])
# Trier les corrélations dans l'ordre décroissant
correlations_sorted = correlations.sort_values(ascending=False)
# Afficher les corrélations triées
#print(correlations_sorted)
# Filtrer pour garder uniquement les corrélations supérieures à 0.1
correlations_filtered = correlations_sorted[correlations_sorted > 0.1]
# Afficher les corrélations filtrées
print(correlations_filtered)

Surface_habitable_logement    0.639725
Ubat_W/m²_K                   0.314571
_score                        0.196218
Code_postal_(BAN)             0.156648
dtype: float64


In [19]:
# Sélectionner les colonnes qualitatives parmi les variables explicatives
qualitative_variables = [col for col in les_variables_explicatives if not np.issubdtype(df1[col].dtype, np.number)]
# Afficher les colonnes qualitatives sélectionnées
#print("Colonnes qualitatives parmi les variables explicatives :", qualitative_variables)
#qualitative_variables

In [20]:
df1['Conso_5_usages_é_finale'].describe()
#Il faut traiter les outliers de df1['Conso_5_usages_é_finale'] pour éviter que le modèle apprenne mal

count    220630.000000
mean      10416.044144
std        9504.693253
min         306.300000
25%        4985.900000
50%        7984.400000
75%       13082.150000
max      828004.300000
Name: Conso_5_usages_é_finale, dtype: float64

In [21]:
EI = 13158.900000 - 5015.400000
limitSup = 13158.900000 + (EI * 1.5)
print(limitSup) #le seuil qui considère valeurs aberrantes niv sup

# Compter le nombre de valeurs supérieures à 20000 (outliers)
count_above_20000 = (df1['Conso_5_usages_é_finale'] > 25000).sum()

# Afficher le résultat
print("Nombre de valeurs supérieures à 25000 :", count_above_20000)

# Calculer le pourcentage
percentage_above_20000 = (count_above_20000 / len(df1['Conso_5_usages_é_finale'])) * 100

# Afficher le résultat
print("Pourcentage de valeurs supérieures à 25000 :", percentage_above_20000, "%")

25374.15
Nombre de valeurs supérieures à 25000 : 11007
Pourcentage de valeurs supérieures à 25000 : 4.988895435797489 %


In [22]:
# Taille du data 
df1.shape

(220630, 103)

In [23]:
df2 = df1
df2 = df2[df2['Conso_5_usages_é_finale'] <= 25000] #suppression des outliers qui représentent 5% de la colonne

In [24]:
"""import scipy.stats as stats
#faire test stat pour évaluer corrélation entre variable cible et les var qualitatitves
qualitative_vars = df2.select_dtypes(include=['object', 'category']).columns
qualitative_vars
quantitative_var = 'Conso_5_usages_é_finale'
# Dictionnaire pour stocker les résultats
anova_results = {}

# Boucle pour effectuer le test ANOVA pour chaque variable qualitative
for qual_var in qualitative_vars:
    # Créer les groupes pour l'ANOVA en fonction des catégories de la variable qualitative
    groups = [df2[df2[qual_var] == category][quantitative_var] for category in df2[qual_var].unique()]
    # Filtrer les groupes pour ne garder que ceux avec au moins deux valeurs
    groups = [group for group in groups if len(group) > 1]

    # Effectuer le test ANOVA si le nombre de groupes est supérieur à 1
    if len(groups) > 1:
        f_stat, p_value = stats.f_oneway(*groups)
        anova_results[qual_var] = {'F_statistic': f_stat, 'p_value': p_value}

# Afficher les résultats
for var, result in anova_results.items():
    print(f"Variable qualitative : {var}")
    print(f"  Statistique F : {result['F_statistic']:.4f}, Valeur p : {result['p_value']:.4e}\n")"""



'import scipy.stats as stats\n#faire test stat pour évaluer corrélation entre variable cible et les var qualitatitves\nqualitative_vars = df2.select_dtypes(include=[\'object\', \'category\']).columns\nqualitative_vars\nquantitative_var = \'Conso_5_usages_é_finale\'\n# Dictionnaire pour stocker les résultats\nanova_results = {}\n\n# Boucle pour effectuer le test ANOVA pour chaque variable qualitative\nfor qual_var in qualitative_vars:\n    # Créer les groupes pour l\'ANOVA en fonction des catégories de la variable qualitative\n    groups = [df2[df2[qual_var] == category][quantitative_var] for category in df2[qual_var].unique()]\n    # Filtrer les groupes pour ne garder que ceux avec au moins deux valeurs\n    groups = [group for group in groups if len(group) > 1]\n\n    # Effectuer le test ANOVA si le nombre de groupes est supérieur à 1\n    if len(groups) > 1:\n        f_stat, p_value = stats.f_oneway(*groups)\n        anova_results[qual_var] = {\'F_statistic\': f_stat, \'p_value\': p_

In [25]:
#choix "au hasard" de variable qualitative pour tester modele
label_encoder = LabelEncoder()
df1['Etiquette_DPE'] = label_encoder.fit_transform(df1['Etiquette_DPE'])

In [26]:
df1['Type_énergie_principale_chauffage'] = label_encoder.fit_transform(df1['Type_énergie_principale_chauffage'])

In [27]:
#df1['Qualité_isolation_murs'] = label_encoder.fit_transform(df1['Qualité_isolation_murs'])

In [28]:
df1['Type_énergie_n°1'] = label_encoder.fit_transform(df1['Type_énergie_n°1'])

### Test Régression

In [29]:
#Test pour régression
#avec seulement les valeurs quantitatives en valeur explicative

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

# Reshape X pour qu'il soit à 2 dimensions
X = df2[['Surface_habitable_logement', 'Ubat_W/m²_K', 'Etiquette_DPE' ,'_score', 'Type_énergie_principale_chauffage', 'Type_énergie_n°1']]  # Utilisez des doubles crochets pour obtenir un DataFrame

Y = df2['Conso_5_usages_é_finale']

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Modèle de régression linéaire
lr_model = LinearRegression()
lr_model = lr_model.fit(X_train, y_train)

y_pred = lr_model.predict(X_test)

print("MAE : " + str(mean_absolute_error(y_test, y_pred)))
print("RMSE : " + str(root_mean_squared_error(y_test, y_pred)))
print("R² : " + str(r2_score(y_test, y_pred)))


ValueError: could not convert string to float: 'D'

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Créer et entraîner un modèle de forêt aléatoire
rf_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
rf_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Calcul des métriques
print("RMSE : " + str(root_mean_squared_error(y_test, y_pred)))
print("MAE : " + str(mean_absolute_error(y_test, y_pred)))
print("R2 : " + str(r2_score(y_test, y_pred)))

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

# Entraîner le modèle
my_model = XGBRegressor()
my_model.fit(X_train, y_train)

# Prédictions sur les ensembles de test et de train
y_pred_test = my_model.predict(X_test)
y_pred_train = my_model.predict(X_train)

# Calcul des métriques pour l'ensemble de test
rmse_test = root_mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

# Calcul des métriques pour l'ensemble de train
rmse_train = root_mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

# Affichage des métriques
print("Métriques sur l'ensemble de Test :")
print("RMSE :", rmse_test)
print("MAE :", mae_test)
print("R2 :", r2_test)

print("\nMétriques sur l'ensemble de Train :")
print("RMSE :", rmse_train)
print("MAE :", mae_train)
print("R2 :", r2_train)


## Features importances avec Random Forest

In [ ]:
# {'Ubat_W/m²_K', "Complément_d'adresse_bâtiment", 'Méthode_application_DPE', 'N°_département_(BAN)', 'Qualité_isolation_menuiseries', 'Nombre_appartement', 'Emission_GES_5_usages_énergie_n°2', 'Coordonnée_cartographique_X_(BAN)', 'Date_fin_validité_DPE', 'Protection_solaire_exterieure_(0/1)', 'Coût_total_5_usages', 'Conso_ECS_dépensier_é_primaire', 'Catégorie_ENR', 'Présence_brasseur_air_(0/1)', 'Date_établissement_DPE', 'Score_BAN', 'Type_énergie_n°2', 'Conso_auxiliaires_é_primaire', 'N°_étage_appartement', 'N°_région_(BAN)', 'Deperditions_planchers_hauts', 'Adresse_brute', 'Emission_GES_refroidissement_dépensier', 'Version_DPE', 'Deperditions_planchers_bas', 'Surface_tertiaire_immeuble', 'Coût_chauffage', '_i', 'Qualité_isolation_plancher_bas', 'Déperditions_renouvellement_air', 'Surface_habitable_logement', 'Type_énergie_n°1', 'Conso_chauffage_é_finale', 'Besoin_refroidissement', 'Emission_GES_éclairage', 'Code_INSEE_(BAN)', 'Conso_refroidissement_dépensier_é_primaire', 'Conso_ECS_dépensier_é_finale', 'Typologie_logement', '_score', 'Classe_altitude', 'Coût_ECS', 'Conso_ECS_é_primaire', 'Nombre_niveau_logement', 'Adresse_(BAN)', 'Emission_GES_ECS', 'Conso_chauffage_dépensier_é_finale', 'Date_visite_diagnostiqueur', 'Conso_refroidissement_é_finale', 'Déperditions_ponts_thermiques', 'Coordonnée_cartographique_Y_(BAN)', 'Type_énergie_n°3', '_geopoint', 'Emission_GES_5_usages_par_m²', 'Logement_traversant_(0/1)', 'Conso_chauffage_é_primaire', 'Deperditions_enveloppe', "Cage_d'escalier", 'Emission_GES_5_usages_énergie_n°1', 'Hauteur_sous-plafond', 'Inertie_lourde_(0/1)', 'Qualité_isolation_enveloppe', 'Conso_éclairage_é_finale', 'Identifiant__BAN', 'Déperditions_portes', 'Coût_chauffage_dépensier', 'Emission_GES_auxiliaires', 'Statut_géocodage', 'Emission_GES_refroidissement', 'Position_logement_dans_immeuble', 'Coût_refroidissement', 'Conso_5_usages_é_primaire', 'Besoin_ECS', 'Coût_chauffage_énergie_n°1', 'Indicateur_confort_été', 'Conso_5_usages_é_finale_énergie_n°1', 'Code_postal_(BAN)', 'Emission_GES_chauffage', 'Type_installation_chauffage', '_id', 'Coût_ECS_dépensier', 'Surface_habitable_immeuble', 'Conso_éclairage_é_primaire', '_rand', 'Nombre_niveau_immeuble', 'N°_DPE_remplacé', 'Conso_5_usages/m²_é_finale', 'Type_bâtiment', 'Conso_refroidissement_dépensier_é_finale', 'Nom_résidence', 'Date_réception_DPE', 'Qualité_isolation_plancher_haut_comble_aménagé', 'N°DPE', 'Type_énergie_principale_chauffage', 'Type_installation_ECS_(général)', 'Isolation_toiture_(0/1)', 'Conso_5_usages_é_finale', 'Conso_chauffage_dépensier_é_primaire', 'Coût_éclairage', 'Classe_inertie_bâtiment', 'Etiquette_GES', 'N°_voie_(BAN)', 'Coût_ECS_énergie_n°1', 'Emission_GES_chauffage_dépensier', 'Coût_chauffage_énergie_n°2', 'Conso_auxiliaires_é_finale', 'Déperditions_murs', 'Deperditions_baies_vitrées', 'Qualité_isolation_murs', 'Etiquette_DPE', 'Emission_GES_5_usages', 'Emission_GES_5_usages_énergie_n°3', 'Conso_refroidissement_é_primaire', 'N°_DPE_immeuble_associé', 'Coût_ECS_énergie_n°2', 'Invariant_fiscal_logement', 'Code_postal_(brut)', 'Zone_climatique_', 'Coût_ECS_énergie_n°3', 'Coût_chauffage_énergie_n°3', 'Conso_ECS_é_finale', 'Modèle_DPE', 'Coût_auxiliaires', 'Coût_refroidissement_dépensier', 'Nom__rue_(BAN)', 'Nom__commune_(BAN)', 'Emission_GES_ECS_dépensier', 'Conso_5_usages_par_m²_é_primaire', 'Besoin_chauffage', 'Appartement_non_visité_(0/1)', "Complément_d'adresse_logement"}
# 131
# (287165, 131)
# Liste des colonnes ainsi que la shape
# On veut comme target Etiquette_DPE ainsi que Conso_5_usages_é_finale


### Traitement Classification avec Etiquette_DPE

In [ ]:
# Je pars du principe que df1 est déjà nettoyer

target = "Etiquette_DPE"
print(df1[target].value_counts().sum)
# Il n'y a que 5 étiquuettes + imbalanced data ? Utilisez SMOTE ?

### Label Encoding

In [ ]:
if df1[target].dtype == 'object':
    label_encoder = LabelEncoder()
    df1[target] = label_encoder.fit_transform(df1[target])


### Train test/split

In [ ]:
X = df1.drop(target, axis=1)
y = df1[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Model

In [ ]:
# 3. Encode categorical features in X_train and X_test separately
# Create a single LabelEncoder for each column to ensure consistent encoding across train and test sets

#Store all encoders in a dictionary
encoders = {}

for column in X_train.select_dtypes(include=['object']).columns:
    encoders[column] = LabelEncoder()
    #Fit on the combined data of train and test
    # Handle unseen values during transform by setting 'handle_unknown' to 'ignore'

    # Ensure all unique values in both train and test are considered
    all_values = pd.concat([X_train[column], X_test[column]], axis=0).unique()

    encoders[column].fit(all_values)  # fitting only train data previously

    # Use try-except block to handle unseen values during transform
    try:
        X_train[column] = encoders[column].transform(X_train[column])
        X_test[column] = encoders[column].transform(X_test[column]) # Use transform, not fit_transform, on test dat
    except ValueError as e:
        # Print the error message and the problematic column
        print(f"Error encoding column '{column}': {e}")
        print(f"Dropping column '{column}' due to unseen values.")
        X_train = X_train.drop(columns=[column])
        X_test = X_test.drop(columns=[column])

# Convert all columns to numeric after encoding to avoid data type issues
# errors='coerce' will replace any value that cannot be converted with NaN.
# You may need to handle these NaN values (e.g., imputation) separately if they are significant.

X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

### Evaluation

In [ ]:
y_pred = model.predict(X_test)

# accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy du Random Forest : {accuracy}")


In [ ]:
feature_importances = model.feature_importances_
# feature_importances

In [ ]:

# Tri des features par importance
indices = np.argsort(feature_importances)[::-1]
top_n = 5  # Limite à 5
top_features = X.columns[indices[:top_n]]
top_importances = feature_importances[indices[:top_n]]

# Visualisation
plt.figure(figsize=(10, 8))
plt.barh(range(top_n), top_importances[::-1], align='center')
plt.yticks(range(top_n), top_features[::-1])
plt.xlabel("Importance des variables")
plt.title("Top 5 des variables les plus importantes pour prédire 'Etiquette_DPE'")
plt.show()


In [ ]:

# Définir une fonction pour sauvegarder le modèle
def save_model(model, filename='RF_Classification_Etiquette.pkl'):
    """
    Sauvegarde le modèle au format pickle avec gestion des exceptions.

    Parameters:
    model: objet scikit-learn entraîné
    filename (str): chemin et nom du fichier pour sauvegarder le modèle
    """

    # Sauvegarde avec gestion d'exception
    try:
        with open(filename, 'wb') as file:
            pickle.dump(model, file)
        print(f"Modèle sauvegardé avec succès sous : {filename}")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde du modèle : {e}")

# Appel de la fonction pour sauvegarder le modèle
save_model(model, 'RF_Classification_Etiquette.pkl')
